# Adaptive PDE discretizations on cartesian grids
## Volume : Non-Divergence form PDEs
## Part : Applications
## Chapter : Boat routing


In this notebook, we compute the optimal route for a boat under the following assumptions:
- The objective is to minimize the fuel consumption, for a given the travel time.
- The boat fuel consumption is locally a quadratic function of its relative velocity, w.r.t. water.
- The boat total velocity is the sum of the relative velocity w.r.t water and of a drift due to water currents.

**Discussion of the assumptions**

These assumptions chosen above are a strong simplification of reality. Some of them may be relaxed, at the price of more complexity in the implementation. 
- Often the objective is to minimize fuel consumption subject to a window in the arrival dates.
- The boat velocity model can be complexified. In fact, the method described below can handle any convex cost, approximated in a suitable manner. However, determining this cost (which depends on the physical properties of the boat), and approximating it (fitting a convex set with an intersection of ellipsoids), may raise difficulties in itself. 
- The problem considered here is time dependent. However, if time dependency is not a prerequisite for the application, then a much simpler and faster alternative numerical method can be considered, described in another notebook.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('BoatRouting','FMM'))

In [2]:
from agd import HFMUtils
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd.Metrics import Rander,Riemann
from agd import AutomaticDifferentiation as ad
from agd.Plotting import savefig; #savefig.dirName = 'Images/BoatRouting'

In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

Utility functions

In [49]:
norm_infinity = ad.Optimization.norm_infinity
from agd.ExportedCode.Notebooks_NonDiv.LinearMonotoneSchemes2D import streamplot_ij

def quiver(X,Y,U,V,subsampling=1,**kwargs):
    def f(Z): return Z[::subsampling,::subsampling]
    plt.quiver(f(X),f(Y),f(U),f(V),**kwargs)

## 1. The model